# Experimentations
## Comparison between involution and convolution on CIFAR10 

## Instructions

- Create and checkout a branch for your work (`git checkout -b YOUR_BRANCH_NAME`)
- Create a copy of this template and name it however you want (e.g. `YOUR_NAME.ipynb`).
- Change the title according to what scenario you are testing.
- In the "involution" part, replace the existing model with the relevant model.
- Create as many cells as you deem necessary for the experiments (explain your use case, comparison with convolution or other networks, results, etc.).
- Remove this cell and every "TODO" comment.
- When you are done, commit your changes. Make sure you are only committing changes on the file you created! (you can check with `git status`)
- Push your changes on the repo (`git push --set-upstream origin YOUR_BRANCH_NAME`).
- Create a pull request to the `main` branch so that everyone can read your code before we merge it.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from involution import Involution
import time
import visualkeras

## Import and formatting of data (CIFAR10)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

class_img = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse' , 'ship', 'truck']

num_classes = len(class_img)

batch_size = 256

epochs = 10 # TODO A AUGMENTER

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Creating datasets
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(batch_size).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices( (x_test, y_test)).batch(batch_size)

## Convolution (model 1)

In [ ]:
modelCNN1 = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding="same"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes),
    ]
)

In [ ]:
modelCNN1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

start_time_CNN1 = time.time()

history = modelCNN1.fit(x=x_train, 
          y=y_train, 
          epochs=epochs, 
          validation_data=(x_test, y_test),
          verbose=1)

training_time_CNN1 = time.time() - start_time_CNN1

accuracy_CNN1 = np.max(history.history['val_accuracy'])

In [ ]:
modelCNN1.summary()

In [ ]:
print("Accuracy of CNN model 1 : " + str(accuracy_CNN1))
print("Training time of CNN model 1 : " + str(training_time_CNN1))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,8))

ax[0].plot(history.history['accuracy'], label='training accuracy')
ax[0].plot(history.history['val_accuracy'], label='validation accuracy')
ax[0].grid()
ax[0].legend()
ax[0].set_title('Accuracy vs. Epochs - CNN model 1')
ax[0].set_xlabel('# Epochs')

ax[1].plot(history.history['loss'], label='training loss')
ax[1].plot(history.history['val_loss'], label='validation loss')
ax[1].grid()
ax[1].legend()
ax[1].set_title('Loss vs. Epochs - CNN model 1')
ax[1].set_xlabel('# Epochs')

## Involution (model 1)

In [ ]:
modelINN1 = tf.keras.models.Sequential([
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu1"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu2"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Involution(channel=3,group_number=1,kernel_size=3,stride=1,reduction_ratio=2),
    tf.keras.layers.ReLU(name="relu3"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes),
])

modelINN1.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [ ]:
start_time_INN1 = time.time()

history = modelINN1.fit(x=x_train, 
          y=y_train, 
          epochs=epochs, 
          validation_data=(x_test, y_test),
          verbose=1)

training_time_INN1 = time.time() - start_time_INN1

accuracy_INN1 = np.max(history.history['val_accuracy'])

In [ ]:
print("Accuracy of INN model 1 : " + str(accuracy_INN1))
print("Training time of CINN model 1 : " + str(training_time_INN1))

In [ ]:
modelINN1.summary()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,8))

ax[0].plot(history.history['accuracy'], label='training accuracy')
ax[0].plot(history.history['val_accuracy'], label='validation accuracy')
ax[0].grid()
ax[0].legend()
ax[0].set_title('Accuracy vs. Epochs - INN model 1')
ax[0].set_xlabel('# Epochs')

ax[1].plot(history.history['loss'], label='training loss')
ax[1].plot(history.history['val_loss'], label='validation loss')
ax[1].grid()
ax[1].legend()
ax[1].set_title('Loss vs. Epochs - INN model 1')
ax[1].set_xlabel('# Epochs')

**Comparison CNN | INN** *(Google Colab, with GPU)*:

*   Number of parameters: 319,178 | 13,080
*   Execution time (seconds): 142.63 | 231.14
*   Accuracy: 0.73 | 0.45

## Représentation graphique des modèles

### Convolution

In [ ]:
visualkeras.layered_view(modelCNN1, draw_volume=False, legend=True)

### Involution

In [ ]:
visualkeras.layered_view(modelINN1, draw_volume=False, legend=True)